In [14]:
import cv2
from matplotlib import pyplot as plt
import seaborn as sns
import collections
import numpy as np
import itertools
import glob
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from realsense_detector import detect_contour
import copy
from realsense_detector import main2

%config InlineBackend.figure_format = 'retina'

In [70]:
def count_color(image):
    color_list = []
    for i in range(3):
        color = []
        color_dict = collections.Counter(image[:, :, i].ravel())
        for num in range(256):
            if not num in color_dict:
                color_dict[num] = 0
        for index in sorted(color_dict.keys()):
            color.append(color_dict[index])
        color_list.append(color)
    return color_list

def count_hsv(image):
    hsv_list = []
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    h,s,v = cv2.split(hsv)
    for i, data in enumerate([h, s, v]):
        sort_data = []
        end = 256
        data_dict = collections.Counter(data.ravel())
        if i == 0:
            end = 180
        for num in range(end):
            if not num in data_dict:
                data_dict[num] = 0
        for index in sorted(data_dict.keys()):
            sort_data.append(data_dict[index])
        hsv_list.append(sort_data)
    return hsv_list

def view_color(result_list):
    plt.figure(figsize=(12, 5))
    plt.plot(result_list[0], color='red', label='Red')
    plt.plot(result_list[1], color='green', label='Green')
    plt.plot(reslut_list[2], color='blue', label='Blue')
    plt.legend()
    plt.show()
    
# 深度情報の特徴量を作成する関数
def get_depth_feature(img):
    depth_dict = {}
    depth_list = []
    MAX_DEPTH = 400
    for col in img:
        for num in col:
            if num not in depth_dict:
                depth_dict[num] = 1
            else:
                depth_dict[num] += 1
    
    for num in range(50, MAX_DEPTH):
        if num not in depth_dict:
            depth_list.append(0)
        else:
            depth_list.append(depth_dict[num])
    
    return depth_list


# 色と深度情報を合わせた特徴量のデータフレームを返す関数
def color_and_depth(color_img, depth_img, name):
    result_dict = {}
    RGB = list(itertools.chain.from_iterable(count_color(color_img)))
    HSV = list(itertools.chain.from_iterable(count_hsv(color_img)))
    Depth = get_depth_feature(depth_img)
    features = RGB + HSV + Depth
    result_dict[name] = features
    df = pd.DataFrame.from_dict(result_dict, orient='index')
    return df

In [147]:
df = pd.read_csv('base_before.csv')
#df1 = pd.read_csv('base.csv')
df1

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_1801,feature_1802,feature_1803,feature_1804,feature_1805,feature_1806,feature_1807,feature_1808,feature_1809,y
0,1400,277,238,193,242,166,191,173,181,197,...,0,0,0,0,0,0,0,0,0,iris
1,0,0,0,0,0,0,0,0,0,0,...,124,483,1008,610,70,0,0,0,0,iris
2,3,5,2,7,13,14,22,26,38,39,...,0,0,0,0,0,0,0,0,0,iris


In [3]:
paths = glob.glob('pan_test_data/train/*')

In [10]:
def get_color_feature(paths):
    result_dict = {}
    for path in paths:
        name = path[path.find('/train/')+7:path.find('.')]
        img = cv2.imread(path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        RGB = list(itertools.chain.from_iterable(count_color(img)))
        HSV = list(itertools.chain.from_iterable(count_hsv(img)))
        color_feature = RGB + HSV
        result_dict[name] = color_feature
    df = pd.DataFrame.from_dict(result_dict, orient='index')
    return df

In [11]:
df = get_color_feature(paths)

In [6]:
train_y = df.index
le = LabelEncoder()
le = le.fit(train_y)
train_y_ = le.transform(train_y)
goods_list = list(le.classes_)
goods_dict = {}
for i, name in enumerate(goods_list):
    goods_dict[i] = name
# y_train = np_utils.to_categorical(train_y_)
y_train = train_y_
X_train = df.values
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=1)

In [7]:
test_path = glob.glob('pan_test_data/test/*')
test = get_color_feature(test_path).values
predicts = knn.predict(test)
match = 0
all_len = len(predicts)
for i, pre in enumerate(predicts):
    acc_name = test_path[i][test_path[i].find('/test/')+6:test_path[i].find('.')][:-5]
    pre_name = goods_dict[pre][:-6]
    text_acc = 'acc: {}'.format(acc_name)
    text_name = 'pre: {}'.format(pre_name)
    text_bar = '-'*30+'\n'
    print('acc:', acc_name)
    print('pre:', pre_name)
    if acc_name == pre_name:
        match += 1
    print('-'*30)
    with open('color_result.txt', 'a') as f:
        f.write(text_acc+'\n')
        f.write(text_name+'\n')
        f.write(text_bar)
    
print('Accuracy Rate:', str(round(match/all_len*100, 1))+'%', '('+str(match)+'/'+str(all_len)+')')
text_result = 'Accuracy Rate: {} {}'.format(str(round(match/all_len*100, 1))+'%', '('+str(match)+'/'+str(all_len)+')')
with open('HSV_result.txt', 'a') as f:
    f.write(text_result)

acc: curry
pre: curry
------------------------------
acc: cream
pre: cream
------------------------------
acc: kurumiC
pre: kurumiC
------------------------------
acc: sausageB
pre: sausageB
------------------------------
acc: kurumiB
pre: kurumiA
------------------------------
acc: allegra
pre: allegra
------------------------------
acc: curry_stick
pre: curry_stick
------------------------------
acc: kurumiA
pre: kurumiA
------------------------------
acc: apple
pre: apple
------------------------------
acc: dog
pre: allegra
------------------------------
acc: sausageA
pre: sausageA
------------------------------
acc: realsense
pre: realsense
------------------------------
acc: whitesource
pre: whitesource
------------------------------
Accuracy Rate: 84.6% (11/13)
